In [14]:
import pandas as pd
import numpy as np
import os
import json
import xgboost as xgb
from tqdm import tqdm
import sklearn.metrics
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.externals import joblib
from F1Optimizer import F1Optimizer
import random
import time
tqdm.pandas()
from sklearn.metrics import f1_score

### Final File
<p>The final file has been created by keeping the production of model in mind by that i mean is we would take very less number parameter from the user and there are some parameters like days_since_prior_order, order_number that we have choosen randomly and also there are some parameters like order_dow, order_hour_of_day that we have taken from current timestamp.</p>
<p>In final file i have created two function - Prediction(), Prediction_with_Metric()
<p>In Prediction(), we take just a user_id as input and output the products that need to be recommended.</p>
<p>In Prediction_with_Metric(), i take list of users and list of actual produts that they are gonne order and the output for this is mean f1 score to get rough idea of how model performs.</p>
<p>I am not predicting None in the Production model.As i think it would look a little bit odd to user.</p>

In [10]:
def Prediction(user_id):
    pred=[]
    frst1 = time.time()
    dow = datetime.now().month
    hour = datetime.now().hour
    here1 = joblib.load('final_sub/user_prod_orderdow.pkl')
    here1 = here1[here1['user_id']==user_id]
    if here1.shape[0]!=0:
        here1 = here1[here1['user_id']==user_id]
        list_days = joblib.load('final_sub/list_days.pkl')
        products = joblib.load('final_sub/products.pkl')
        orders_prior_department_reordered = joblib.load('final_sub/orders_prior_department_reordered.pkl')
        orders_prior_aisle_reordered = joblib.load('final_sub/orders_prior_aisle_reordered.pkl')
        productFeat = joblib.load('final_sub/productFeat.pkl')
        user_feat1 = joblib.load('final_sub/user_feat1.pkl')
        product_features1 = joblib.load('final_sub/product_features1.pkl')
        user_product_order = joblib.load('final_sub/user_product_order.pkl')
        temp_data = joblib.load('final_sub/order_prod_addToCart.pkl')
        product_embeddings = joblib.load('final_sub/product_embeddings.pkl')
    
        
        def get_order_dow(x):
            if any(x['order_dow']==dow):
                return x[x['order_dow']==dow]
            else:
                a = x[x['user_id']==user_id].head(1)
                a['order_dow']=dow
                a['week_product_reordered']=0
                a['week_product_ordered']=0
                a['week_product_reordered_ration']=0
                return a

        am = here1.groupby('product_id').apply(get_order_dow)
        convert_dict = {'user_id': int,
                        'product_id': int,
                        'order_dow': int,
                        'week_product_reordered': int,
                        'week_product_ordered': int,
                        'week_product_reordered_ration': float,
                       }
        data = am.astype(convert_dict)
        data.reset_index(drop=True, inplace=True) 
        data = data.merge(list_days,on=["user_id", 'product_id'])
        # del list_days
        data = data.merge(products,on="product_id")
        data = data.merge(orders_prior_department_reordered,on=['user_id', 'department_id'])
        data = data.merge(orders_prior_aisle_reordered,on=['user_id', 'aisle_id'])

        data = data.merge(productFeat,on='product_id')
        data = data.merge(user_feat1,on='user_id')
        data = data.merge(product_features1,on='product_id')
        data = data.merge(user_product_order,on=['user_id','product_id'])
        data = data.merge(temp_data,on=['user_id','product_id'])
        data = data.merge(product_embeddings,on='product_id')

        del list_days, products, orders_prior_department_reordered, orders_prior_aisle_reordered, productFeat, user_feat1, product_features1, user_product_order, temp_data, product_embeddings

        # gave random value for some of the parameters
        data['days_since_prior_order']=float(random.randint(1, 30))
        data['order_number']=random.randint(10, 20)
        data['order_hour_of_day']=hour

        test = data[['product_id', 'user_id', 'order_number', 'order_dow',
           'order_hour_of_day', 'days_since_prior_order', 'aisle_id',
           'department_id', 'unique_prod_in_aisle', 'aisle_product_reordered',
           'unique_prod_in_department', 'department_product_reordered',
           'number_of_unique_users_for_product',
           'number_of_unique_users_for_product_reordered', '0', '1', '2', '3', '4',
           '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
           '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
           '30', '31', 'days_before_product_ordered-1',
           'days_before_product_ordered-4', 'days_before_product_ordered-2',
           'days_before_product_ordered_mean', 'days_before_product_ordered-3',
           'days_before_product_ordered_median', 'prod_orders',
           'prod_ordersBYtotal_orders', 'product_ordered_last',
           'product_ordered_first', 'prod_min_orderNumber', 'prod_max_orderNumber',
           'prod_cart_mean', 'days_since_prior_order_mean', 'order_dow_mean',
           'order_hour_of_day_mean', 'add_to_cart_order_inverse_mean',
           'add_to_cart_order_relative_mean', 'reordered_sum',
           'week_product_reordered', 'week_product_ordered',
           'week_product_reordered_ration', 'user_orders_count', 'user_since',
           'user_mean_days_since_prior_order',
           'user_median_days_since_prior_order', 'unique_prod',
           'NumberTimeProductsOrdered', 'user_reorder_ratio',
           'Avergae_Basket_size']]

        bst = joblib.load('final_sub/model.pkl') 
        pred_Xcv = bst.predict(xgb.DMatrix(test))

        data['pred'] = pred_Xcv
        test_pred = data[['user_id','product_name','product_id','pred']].groupby('user_id').agg({'product_id': list,'product_name': list, 'pred': list})

        def sort_values(row):  
            ok = np.argsort(np.array(row['pred']))
            test_pred.at[row.name,'product_id']=[row['product_id'][i] for i in ok][::-1]
            test_pred.at[row.name,'product_name']=[row['product_name'][i] for i in ok][::-1]
            test_pred.at[row.name,'pred'] = [row['pred'][i] for i in ok][::-1]

        test_pred.apply(sort_values,axis=1)

        for index, row in test_pred.iterrows():
            opt=F1Optimizer.maximize_expectation(row.pred)
            best_k=opt[0]
            pred=test_pred[test_pred.index==index]['product_id'].values[0][:best_k]
    else:
        Top_prod_DOW_Hour = joblib.load('final_sub/Top_prod_DOW_Hour.pkl')
        pred = Top_prod_DOW_Hour[(Top_prod_DOW_Hour['order_dow']==dow) & (Top_prod_DOW_Hour['order_hour_of_day']==hour)]['top_prod'].values[0]
        if not pred:
            Top_prod_Hour = joblib.load('final_sub/Top_prod_Hour.pkl')
            pred = Top_prod_Hour[Top_prod_DOW_Hour['order_hour_of_day']==hour]['top_prod'].values[0]
        
    print('Time it took to Predct(sec): ', time.time() - frst1)
    return pred

In [11]:
products_predicted = Prediction(2)
products_predicted

Time it took to Predct(sec):  18.550846338272095


[24852,
 47209,
 18523,
 16589,
 19156,
 32792,
 1559,
 33754,
 21709,
 22825,
 39928,
 7781,
 20785,
 19057,
 42342,
 45948,
 27413,
 22124]

In [22]:
def Prediction_with_Metric(user_ids,actual_prods):
    predict_prods = []
    pred = []
    frst1 = time.time()
    dow = datetime.now().month
    hour = datetime.now().hour
    here1 = joblib.load('final_sub/user_prod_orderdow.pkl')
    list_days = joblib.load('final_sub/list_days.pkl')
    products = joblib.load('final_sub/products.pkl')
    orders_prior_department_reordered = joblib.load('final_sub/orders_prior_department_reordered.pkl')
    orders_prior_aisle_reordered = joblib.load('final_sub/orders_prior_aisle_reordered.pkl')
    productFeat = joblib.load('final_sub/productFeat.pkl')
    user_feat1 = joblib.load('final_sub/user_feat1.pkl')
    product_features1 = joblib.load('final_sub/product_features1.pkl')
    user_product_order = joblib.load('final_sub/user_product_order.pkl')
    temp_data = joblib.load('final_sub/order_prod_addToCart.pkl')
    product_embeddings = joblib.load('final_sub/product_embeddings.pkl')
    for user_id in user_ids:
        her = here1[here1['user_id']==user_id]
        if here1.shape[0]!=0:
            her = here1[here1['user_id']==user_id]

            def get_order_dow(x):
                if any(x['order_dow']==dow):
                    return x[x['order_dow']==dow]
                else:
                    a = x[x['user_id']==user_id].head(1)
                    a['order_dow']=dow
                    a['week_product_reordered']=0
                    a['week_product_ordered']=0
                    a['week_product_reordered_ration']=0
                    return a

            am = her.groupby('product_id').apply(get_order_dow)
            convert_dict = {'user_id': int,
                            'product_id': int,
                            'order_dow': int,
                            'week_product_reordered': int,
                            'week_product_ordered': int,
                            'week_product_reordered_ration': float,
                           }
            data = am.astype(convert_dict)
            data.reset_index(drop=True, inplace=True) 
            data = data.merge(list_days,on=["user_id", 'product_id'])
            data = data.merge(products,on="product_id")
            data = data.merge(orders_prior_department_reordered,on=['user_id', 'department_id'])
            data = data.merge(orders_prior_aisle_reordered,on=['user_id', 'aisle_id'])

            data = data.merge(productFeat,on='product_id')
            data = data.merge(user_feat1,on='user_id')
            data = data.merge(product_features1,on='product_id')
            data = data.merge(user_product_order,on=['user_id','product_id'])
            data = data.merge(temp_data,on=['user_id','product_id'])
            data = data.merge(product_embeddings,on='product_id')

            # gave random value for some of the parameters
            data['days_since_prior_order']=float(random.randint(1, 30))
            data['order_number']=random.randint(10, 20)
            data['order_hour_of_day']=hour

            test = data[['product_id', 'user_id', 'order_number', 'order_dow',
               'order_hour_of_day', 'days_since_prior_order', 'aisle_id',
               'department_id', 'unique_prod_in_aisle', 'aisle_product_reordered',
               'unique_prod_in_department', 'department_product_reordered',
               'number_of_unique_users_for_product',
               'number_of_unique_users_for_product_reordered', '0', '1', '2', '3', '4',
               '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
               '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
               '30', '31', 'days_before_product_ordered-1',
               'days_before_product_ordered-4', 'days_before_product_ordered-2',
               'days_before_product_ordered_mean', 'days_before_product_ordered-3',
               'days_before_product_ordered_median', 'prod_orders',
               'prod_ordersBYtotal_orders', 'product_ordered_last',
               'product_ordered_first', 'prod_min_orderNumber', 'prod_max_orderNumber',
               'prod_cart_mean', 'days_since_prior_order_mean', 'order_dow_mean',
               'order_hour_of_day_mean', 'add_to_cart_order_inverse_mean',
               'add_to_cart_order_relative_mean', 'reordered_sum',
               'week_product_reordered', 'week_product_ordered',
               'week_product_reordered_ration', 'user_orders_count', 'user_since',
               'user_mean_days_since_prior_order',
               'user_median_days_since_prior_order', 'unique_prod',
               'NumberTimeProductsOrdered', 'user_reorder_ratio',
               'Avergae_Basket_size']]

            bst = joblib.load('final_sub/model.pkl') 
            pred_Xcv = bst.predict(xgb.DMatrix(test))

            data['pred'] = pred_Xcv
            test_pred = data[['user_id','product_name','product_id','pred']].groupby('user_id').agg({'product_id': list,'product_name': list, 'pred': list})

            def sort_values(row):  
                ok = np.argsort(np.array(row['pred']))
                test_pred.at[row.name,'product_id']=[row['product_id'][i] for i in ok][::-1]
                test_pred.at[row.name,'pred'] = [row['pred'][i] for i in ok][::-1]

            test_pred.apply(sort_values,axis=1)

            for index, row in test_pred.iterrows():
                opt=F1Optimizer.maximize_expectation(row.pred)
                best_k=opt[0]
                pred=test_pred[test_pred.index==index]['product_id'].values[0][:best_k]
        else:
            Top_prod_DOW_Hour = joblib.load('final_sub/Top_prod_DOW_Hour.pkl')
            pred = Top_prod_DOW_Hour[(Top_prod_DOW_Hour['order_dow']==dow) & (Top_prod_DOW_Hour['order_hour_of_day']==hour)]['top_prod'].values[0]
            if not pred:
                Top_prod_Hour = joblib.load('final_sub/Top_prod_Hour.pkl')
                pred = Top_prod_Hour[Top_prod_DOW_Hour['order_hour_of_day']==hour]['top_prod'].values[0]
        predict_prods.append(pred)
    def f1_score_single(y_true, y_pred):
        y_true = set(y_true)
        y_pred = set(y_pred)
        cross_size = len(y_true & y_pred)
        if cross_size == 0: return 0.
        p = 1. * cross_size / len(y_pred)
        r = 1. * cross_size / len(y_true)
        return 2 * p * r / (p + r)

    def f1_score(y_true, y_pred):
        return np.mean([f1_score_single(x, y) for x, y in zip(y_true, y_pred)])
    mean_f1score = f1_score(actual_prods, predict_prods)
    return mean_f1score

In [23]:
user_ids=[1,5,13]
actual_prods=[[196, 46149, 25133,49235,38928,26088,10258,39657,26405,13032],[40706,21616,15349,21413],[27086,4210,1199345937,27435]]
mean_f1score= Prediction_with_Metric(user_ids,actual_prods)
mean_f1score

0.5529411764705882